##### Задача 3.10. 
###### Классификация государств по социально-экономическим и демографическим признакам.
###### Имеются социально-экономические и демографические показатели, характеризующие 106 государств мира, принадлежащих к определенной экономической группе (табл. 3.41).


In [ ]:
import pandas as pd
import pyreadstat
data = pd.read_excel('2.1.4. Данные для д.з. №2.xls', sheet_name='Задача 10')

data = data.drop(index=[0, 1]) 

data.columns = data.iloc[0] 
data = data[1:] 

data.reset_index(drop=True, inplace=True)
data.set_index('Страна', inplace=True)
data = data.drop(data.columns[-2:], axis=1)

pyreadstat.write_sav(data, 'spss_data.sav')

data

2,Население,Плотность,ГН,СПЖЖ,СПЖМ,ДС,ВВП,ЭГ,СКД
Страна,,,,,,,,,
Афганистан,20500,25,18,44,45,168,205,3,6.9
Аргентина,33900,12,86,75,68,25.6,3408,6,2.8
Армения,3700,126,68,75,68,27,5000,5,3.2
Австралия,17800,2.3,85,80,74,7.3,16848,1,1.9
Австрия,8000,94,58,79,73,6.7,18396,1,1.5
...,...,...,...,...,...,...,...,...,...
Уругвай,3200,18,89,77,71,17,3131,6,2.4
Узбекистан,22600,50,41,72,65,53,1350,5,3.7
Венесуэла,20600,22,91,76,70,28,2829,6,3.1


##### Задание 1
###### провести кластеризацию государств на оптимальное число кластеров;

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
X = data

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

inertia = []
K_range = range(1, 11)  
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(K_range, inertia, marker='o')
plt.title('Метод локтя для определения оптимального числа кластеров')
plt.xlabel('Число кластеров')
plt.ylabel('Инерция')
plt.xticks(K_range)
plt.grid()
plt.show()